# test데이터에 대해 전처리 진행

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
base_path = './preprocessed data/'
test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(base_path + 'new_test_quality.csv', parse_dates=['time'])
train = pd.read_csv('./training set/train_case1.csv')

# feature engineering

features: train.columns

In [3]:
df1 = test_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1


In [7]:
train.columns

Index(['counts', 'model_nm_model_0', 'model_nm_model_1', 'model_nm_model_2',
       'model_nm_model_3', 'model_nm_model_4', 'model_nm_model_5',
       'model_nm_model_6', 'model_nm_model_7', 'model_nm_model_8',
       'fwver_03.11.1141', 'fwver_03.11.1149', 'fwver_04.16.2641',
       'fwver_04.16.3345', 'fwver_04.16.3439', 'fwver_04.16.3569',
       'fwver_04.16.3571', 'fwver_04.22.1442', 'fwver_04.22.1656',
       'fwver_04.22.1666', 'fwver_04.22.1684', 'fwver_04.22.1778',
       'fwver_04.33.1095', 'fwver_04.33.1125', 'fwver_04.33.1149',
       'fwver_04.33.1171', 'fwver_04.73.2571', 'fwver_04.82.1730',
       'fwver_05.15.2090', 'fwver_05.15.2092', 'fwver_05.15.2114',
       'fwver_05.15.2120', 'fwver_05.15.2122', 'fwver_05.15.3104',
       'fwver_05.66.3571', 'fwver_8.5.3', 'errtype_1', 'errtype_2',
       'errtype_3', 'errtype_5', 'errtype_6', 'errtype_8', 'errtype_9',
       'errtype_10', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_17',
       'errtype_18', 'errtype_19', '

In [4]:
# counts column 추가
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,counts
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2,1
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1,1
...,...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0,1
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1,1


In [8]:
# multiple_model
model_df =  test_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
30000,1
30001,0
30002,0
30003,0
30004,1
30005,0
30006,0


In [16]:
# quality_sum
quality_df = pd.DataFrame({'user_id': np.arange(30000, 44999)})

for i in test_quality.columns[3:]:
    quality_group_df = test_quality.groupby('user_id')[i].sum().to_frame().reset_index()
    quality_df = pd.merge(quality_df, quality_group_df, how='left', on='user_id')
    quality_df = quality_df.fillna(0)
    
# 모두 0값인 quality_3, quality_4 drop
quality_df.set_index(quality_df['user_id'], inplace=True)
quality_df = quality_df.drop(columns=['user_id', 'quality_3', 'quality_4'])

quality_df.head()

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,,
30000,0.0,0.0,0.0,2.0,5.0,10.0,0.0,0.0,4.0,0.0,0.0
30001,-1.0,-1.0,-1.0,2561.0,-1.0,0.0,0.0,0.0,10245.0,-1.0,0.0
30002,-3.0,-3.0,-3.0,130.0,-3.0,0.0,0.0,0.0,384.0,-3.0,0.0
30003,0.0,0.0,0.0,9.0,220.0,560.0,0.0,0.0,30.0,0.0,0.0
30004,-1.0,-1.0,-1.0,1.0,32.0,90.0,0.0,0.0,4.0,-1.0,0.0


In [18]:
# 1. one-hot-encoding
df1.drop(columns=['time', 'errcode'], inplace=True)
df1 = pd.get_dummies(df1, columns=['model_nm', 'fwver', 'errtype'])
df1

,user_id,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15527216,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15527217,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15527218,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15527219,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# period
test_err['day'] = test_err.time.dt.day
test_err['hour'] = test_err.time.dt.hour
test_err.head()


,user_id,time,model_nm,fwver,errtype,errcode,day,hour
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1,1,3
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2,1,3
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1,1,3
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1,1,3
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1,1,3


In [23]:
user_day_df = test_err.groupby('user_id')['day'].unique().to_frame().reset_index()
user_day_df['period'] = 0 

for i in range(len(user_day_df.index)):
   user_day_df['period'][i] = len(user_day_df['day'][i])

user_day_df

,user_id,day,period
0,30000,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",29
1,30001,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",28
2,30002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
3,30003,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",28
4,30004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
...,...,...,...
14993,44994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14994,44995,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14995,44996,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14996,44997,"[1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",28


In [24]:
user_day_df.index = user_day_df['user_id']
user_day_df.drop(columns=['user_id', 'day'], inplace=True)

user_day_df

,period
user_id,
30000,29
30001,28
30002,30
30003,28
30004,30
...,...
44994,30
44995,30
44996,30


In [27]:
# 모든 column 합쳐주기
df1 = df1.groupby('user_id').sum()
dfs = [df1, user_has_multiple_models, quality_df, user_day_df]
dfs = dfs[0].join(dfs[1:])
dfs

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,period
user_id,,,,,,,,,,,,,,,,,,,,,
30000,2731.0,0.0,2669.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,5.0,10.0,0.0,0.0,4.0,0.0,0.0,29.0
30001,284.0,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,2561.0,-1.0,0.0,0.0,0.0,10245.0,-1.0,0.0,28.0
30002,903.0,903.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.0,130.0,-3.0,0.0,0.0,0.0,384.0,-3.0,0.0,30.0
30003,328.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,220.0,560.0,0.0,0.0,30.0,0.0,0.0,28.0
30004,871.0,0.0,622.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,32.0,90.0,0.0,0.0,4.0,-1.0,0.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,1115.0,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,-1.0,0.0,0.0,0.0,4.0,-1.0,0.0,30.0
44995,515.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
44996,1915.0,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.0,31.0,-2.0,0.0,0.0,0.0,109.0,-2.0,0.0,30.0


In [29]:
# 전처리 완료 후 test set 저장
dfs.to_csv('./training set/test_set.csv')

In [40]:
test = pd.read_csv('./training set/test_set.csv')

In [61]:
# testd에는 없는 fwver 삭제
train.drop(columns=['fwver_04.16.2641', 'fwver_04.16.3345', 'fwver_04.22.1442', 
                    'fwver_04.33.1095', 'fwver_05.15.2090', 'fwver_05.15.2122'], inplace=True)

In [65]:
test_c = list(test.columns)
train_c = list(train.columns)

In [66]:
d_columns = []
for i in test_c:
    if i not in train_c:
        d_columns.append(i)
d_columns

['user_id',
 'fwver_03.11.1167',
 'fwver_04.16.3553',
 'fwver_04.22.1170',
 'fwver_04.22.1448',
 'fwver_04.22.1478',
 'fwver_04.22.1608',
 'fwver_04.22.1750',
 'fwver_04.22.1772',
 'fwver_04.33.1185',
 'fwver_04.33.1261',
 'fwver_04.73.2237',
 'fwver_04.73.2569',
 'fwver_04.73.2577',
 'fwver_04.82.1684',
 'fwver_04.82.1778',
 'fwver_05.15.2138',
 'fwver_05.66.3237',
 'fwver_10',
 'fwver_10.22.1770',
 'fwver_10.22.1780',
 'errtype_4',
 'errtype_7',
 'errtype_11',
 'errtype_12',
 'errtype_16',
 'errtype_21',
 'errtype_23',
 'errtype_27',
 'errtype_28',
 'errtype_37',
 'quality_2',
 'quality_7',
 'quality_9',
 'quality_10',
 'period']

In [68]:
test.drop(columns=d_columns, inplace=True)

len(test.columns)

69

In [69]:
test.to_csv('./training set/test.csv', index=False)